<a href="https://colab.research.google.com/github/ryali93/utilitarios/blob/master/extractData/extractdata_senamhi_historic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
e = open("est.json", "r")
data = e.read()
e.close()

import json
est = json.loads(data)
est

In [0]:
import pandas as pd

In [4]:
estaciones = pd.DataFrame(est)
estaciones.head(

,a,b,c,codigo,d,e,f,h,i,j,k,l,latlng,m,n,name,o,p,r,s,t,u,v,w,x,y
0,1,0,0,152204,0,0,0,0,0,0,0,0,"[-5.75619, -78.52306]",0,0,EL PINTOR,0,0,0,0,0,0,0,0,0,0
1,1,0,0,152210,0,0,0,0,0,0,0,0,"[-5.89106, -78.18886]",0,0,MAGUNCHAL,0,0,0,0,0,0,0,0,0,0
2,0,1,0,155105,0,0,0,0,0,0,0,0,"[-10.4, -77.4]",0,0,OCROS,0,0,0,0,0,0,0,0,0,0
3,0,1,0,000441,0,0,0,0,0,0,0,0,"[-9.72919, -77.45365]",0,0,RECUAY,0,0,0,0,0,0,0,0,0,0
4,0,1,0,150903,0,0,0,0,0,0,0,0,"[-10.15775, -77.43222]",0,0,MAYORARCA,0,0,0,0,0,0,0,0,0,0


In [0]:
import requests
from bs4 import BeautifulSoup

In [0]:
response = requests.get("https://web2.senamhi.gob.pe/descarga/?cod=154111")
soup = BeautifulSoup(response.text, "html.parser")

In [0]:
import re
lista = []
for s in soup.find_all('script', {"type":"text/javascript"}):
    lista.append(s.text)

m = lista[1].replace("\n", "").replace("\r", "").replace("\t", "").replace("null", "None")
m

In [0]:

datos = re.findall("\[.*?]", m)
t = [d for d in datos if len(d)>100]
t

In [0]:
# Fechas
d = re.findall("(categories: \[.*?])", t[0])[0]
d = d.split(":")[1][2:-2]
fechas_lista = [int(f) for f in d.replace("'", "").split(",")]
fechas_lista

In [0]:
# Precipitacion
p = re.findall("(data: \[.*?])", t[1])[0]
p = p.split(":")[1][2:-2]

pp_lista = []
for pp in p.split(","):
  if pp == "None":
    pp_lista.append(None)
  else:
    pp_lista.append(float(pp))

In [0]:
# Temp Max
temp = t[2][1:-2]
temp_max_lista = []
for temp in temp.split(","):
  if temp == "None":
    temp_max_lista.append(None)
  else:
    temp_max_lista.append(float(temp))

In [0]:
# Temp Min
temp = t[3][1:-2]
temp_min_lista = []
for temp in temp.split(","):
  if temp == "None":
    temp_min_lista.append(None)
  else:
    temp_min_lista.append(float(temp))

In [0]:
dicc = {
    "fechas": fechas_lista,
    "pp": pp_lista,
    "temp_max": temp_max_lista,
    "temp_min": temp_min_lista,
}

In [0]:
import pandas as pd

In [145]:
df = pd.DataFrame(dicc)
df.head()

,fechas,pp,temp_max,temp_min
18531,2014,0.0,24.6,7.9
18532,2014,0.0,26.4,10.8
18533,2014,0.0,27.0,9.1
18534,2014,0.0,25.3,7.4
18535,2014,0.0,28.0,6.9


In [154]:
df.groupby(["fechas"])["fechas"].count()

fechas
1963     61
1964    366
1965    365
1966    365
1967    365
1968    366
1969    365
1970    365
1971    365
1972    366
1973    365
1974    365
1975    365
1976    366
1977    365
1978    365
1979    365
1980    366
1981    365
1982    365
1983    365
1984    366
1985    365
1986    365
1987    365
1988    366
1989    365
1990    365
1991    365
1992    366
1993    365
1994    365
1995    365
1996    366
1997    365
1998    365
1999    365
2000    366
2001    365
2002    365
2003    365
2004    366
2005    365
2006    365
2007    365
2008    366
2009    365
2010    365
2011    365
2012    366
2013    365
2014    212
Name: fechas, dtype: int64